In [7]:

lyrics = """
(Intro)
Hey, it's KAYPramitO, light the room
We're headed to Florida, in a Zoom

(Verse 1)
Pramit said, "Where do you work? Oh, Wall Street Company!"
Everyone's talking, so much synergy
Neon lights and city nights
Deadlock season's the latest hype

(Chorus)
Let's go shower, flash and tower
30 minutes to power, this is our hour
Grab the orb, shower scene
Cypher’s here, oh how we've been

(Verse 2)
Sova, Reyna, Phoenix too
Headphones on backwards, what a view
Jon says, "Try not to die"
Lucy dings Phoenix, Naiyo in the sky

(Chorus)
Let's go shower, flash and tower
30 minutes to power, this is our hour
Pramit with the AWP, nothing sour
Holding angles, gaming flower

(Bridge)
What’s that dog doing? Jimmy wants to know
Reyna’s lurking, sneaky like a Poe
Get in position, all set, let's roll
Round and round, capture the soul

(Outro)
KAYPramitO, take us out
In this game, we’re never in doubt
Shower's clear, shine bright, no fear
We play for fun, year after year
"""


In [43]:
from moviepy.editor import VideoFileClip, concatenate_videoclips, AudioFileClip
from moviepy.video.fx.loop import loop
import ffmpeg
import asyncio
import time
import os

async def merge_videos_and_song_with_subtitles(srt_path):
    # Step 1: Download song and videos
    # async with ClientSession() as session:
    #     song_path = await download_file(session, song_url, 'song.mp3')
    #     video_paths = []
    #     for i, video_url in enumerate(video_urls):
    #         video_path = await download_file(session, video_url, f'video_{i+1}.mp4')
    #         video_paths.append(video_path)

    song_path = './song.mp3'
    video_paths = [f'./video_{i+1}.mp4' for i in range(6)]

    # Step 2: Load videos using MoviePy
    video_clips = [VideoFileClip(video) for video in video_paths]
    
    # Step 3: Concatenate and loop the videos until they match the song duration
    concatenated_clip = concatenate_videoclips(video_clips)

    song_audio = AudioFileClip(song_path)
    song_duration = song_audio.duration
    final_video_clip = loop(concatenated_clip, duration=song_duration)
    final_video_clip = final_video_clip.set_audio(song_audio)

    # Step 4: Add subtitles using ffmpeg (MoviePy)
    # MoviePy uses ffmpeg to overlay the subtitles without ImageMagick.
    final_video_with_subtitles = final_video_clip.subclip(0, song_duration).set_duration(song_duration).set_audio(song_audio)

    # Step 5: Write the output to a file
    output_path = f"final_output_with_subtitles_{time.time()}.mp4"
    final_video_with_subtitles.write_videofile(output_path, codec='libx265', audio_codec='aac', ffmpeg_params=['-vf', f"subtitles={srt_path}", '-crf', '28'])

    # Step 6: Clean up temporary files
    song_audio.close()
    final_video_clip.close()
    concatenated_clip.close()
    final_video_with_subtitles.close()
    for video in video_clips:
        video.close()
    
    return output_path

In [44]:
await merge_videos_and_song_with_subtitles('./output.ass')


Moviepy - Building video final_output_with_subtitles_1729416223.7528567.mp4.
MoviePy - Writing audio in final_output_with_subtitles_1729416223.7528567TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video final_output_with_subtitles_1729416223.7528567.mp4



Moviepy - Done !
Moviepy - video ready final_output_with_subtitles_1729416223.7528567.mp4


'final_output_with_subtitles_1729416223.7528567.mp4'

In [4]:
import stable_whisper

In [5]:
model = stable_whisper.load_model("base")

In [8]:
# remove everything in brackets of any sory from lyrics
import re
lyrics = re.sub(r'\[.*?\]|\(.*?\)', '', lyrics)


In [37]:
result = model.align('./song.mp3', lyrics, language='en', fast_mode=True, regroup=False)


Align: 100%|██████████| 197.42/197.42 [00:00<00:00, 248.88sec/s]
Adjustment: 100%|██████████| 83.78/83.78 [00:00<00:00, 33514.43sec/s]


In [39]:
result

In [40]:
for segment in result:
    print("[%.2f -> %.2f] %s" % (segment.start, segment.end, segment.text))



[0.00 -> 83.78]   Hey, it's KAYPramitO, light the room We're headed to Florida, in a Zoom   Pramit said, "Where do you work? Oh, Wall Street Company!" Everyone's talking, so much synergy Neon lights and city nights Deadlock season's the latest hype   Let's go shower, flash and tower 30 minutes to power, this is our hour Grab the orb, shower scene Cypher’s here, oh how we've been   Sova, Reyna, Phoenix too Headphones on backwards, what a view Jon says, "Try not to die" Lucy dings Phoenix, Naiyo in the sky   Let's go shower, flash and tower 30 minutes to power, this is our hour Pramit with the AWP, nothing sour Holding angles, gaming flower   What’s that dog doing? Jimmy wants to know Reyna’s lurking, sneaky like a Poe Get in position, all set, let's roll Round and round, capture the soul   KAYPramitO, take us out In this game, we’re never in doubt Shower's clear, shine bright, no fear We play for fun, year after year 


In [41]:
srt = result.to_ass('output.ass')


Saved: c:\Users\pmazu\Documents\GitHub\BlinkStory\output.ass


In [6]:
import os
import google.auth
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
from googleapiclient.http import MediaFileUpload
from google.oauth2.credentials import Credentials

class YouTubeUploader:
    def __init__(self, client_secret_file, api_service_name="youtube", api_version="v3", scopes=None):
        if scopes is None:
            scopes = ["https://www.googleapis.com/auth/youtube.upload"]
        
        self.client_secret_file = client_secret_file
        self.api_service_name = api_service_name
        self.api_version = api_version
        self.scopes = scopes
        self.credentials = None
        self.youtube = None
        self.token_file = 'token.json'

    def authenticate(self):
        # OAuth 2.0 flow to get authenticated credentials
        if os.path.exists('token.json'):
            self.credentials = Credentials.from_authorized_user_file('token.json', self.scopes)
                
        # If no valid credentials, perform the OAuth flow
        if not self.credentials or not self.credentials.valid:
            if self.credentials and self.credentials.expired and self.credentials.refresh_token:
                # Refresh the credentials if they're expired
                self.credentials.refresh(google.auth.transport.requests.Request())
            else:
                # OAuth 2.0 flow to get authenticated credentials for the first time
                flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
                    self.client_secret_file, self.scopes
                )
                self.credentials = flow.run_local_server()

            # Save the credentials to a file for the next time
            with open(self.token_file, 'w') as token:
                token.write(self.credentials.to_json())
        
        # Build the YouTube service object
        self.youtube = googleapiclient.discovery.build(
            self.api_service_name, self.api_version, credentials=self.credentials
        )

    def upload_video(self, video_path, title, description, tags=None, category_id="22", privacy_status="private"):
        if tags is None:
            tags = []

        # Set up the video upload details
        body = {
            "snippet": {
                "title": title,
                "description": description,
                "tags": tags,
                "categoryId": category_id  # Category: 22 is for 'People & Blogs'
            },
            "status": {
                "privacyStatus": privacy_status  # "public", "private", or "unlisted"
            }
        }

        # Upload the video file using the YouTube Data API
        media = MediaFileUpload(video_path, chunksize=-1, resumable=True)

        request = self.youtube.videos().insert(
            part="snippet,status",
            body=body,
            media_body=media
        )

        response = None
        while response is None:
            status, response = request.next_chunk()
            if status:
                print(f"Uploaded {int(status.progress() * 100)}%")

        print("Upload complete!")
        print(f"Video ID: {response['id']}")
        return response

In [8]:

uploader = YouTubeUploader("client_secret.json")
uploader.authenticate()

In [9]:
uploader.upload_video(
    video_path="./final_output.mp4",
    title="Your Video Title",
    description="Description of your video",
    tags=["sample", "video"],
    category_id="22",  # Category ID for 'People & Blogs'
    privacy_status="private"
)

Upload complete!
Video ID: okXnGcHslyc


{'kind': 'youtube#video',
 'etag': 'm3oklTOzemlDpsolC95uDzEHTDI',
 'id': 'okXnGcHslyc',
 'snippet': {'publishedAt': '2024-10-20T09:46:43Z',
  'channelId': 'UCLVW6uqN6az8ReX015UkpCA',
  'title': 'Your Video Title',
  'description': 'Description of your video',
  'thumbnails': {'default': {'url': 'https://i9.ytimg.com/vi/okXnGcHslyc/default.jpg?sqp=CJyi07gG&rs=AOn4CLAh9GoWOYBWNLPnWdmUH_-x31uiVA',
    'width': 120,
    'height': 90},
   'medium': {'url': 'https://i9.ytimg.com/vi/okXnGcHslyc/mqdefault.jpg?sqp=CJyi07gG&rs=AOn4CLDcOV__AKaVnESXLYpJPdG5nKxmWw',
    'width': 320,
    'height': 180},
   'high': {'url': 'https://i9.ytimg.com/vi/okXnGcHslyc/hqdefault.jpg?sqp=CJyi07gG&rs=AOn4CLBfSjzKjP5dsKMbR470bdeNXmpRsw',
    'width': 480,
    'height': 360}},
  'channelTitle': 'Pramit Mazumder',
  'tags': ['sample', 'video'],
  'categoryId': '22',
  'liveBroadcastContent': 'none',
  'localized': {'title': 'Your Video Title',
   'description': 'Description of your video'}},
 'status': {'uploadSta

In [10]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "Automatically generate a YouTube video title, description, and tags based on provided song lyrics and scene descriptions.\n\nUse the given lyrics and scene descriptions to create engaging and relevant content reflecting the theme, mood, and key elements of the input. Ensure the title is compelling, the description is informative, and the tags are relevant to the content.\n\n# Steps\n\n1. **Title Generation**:\n   - Identify key themes, motifs, or unique elements within the input such as \"Neon lights\", \"Digital battlefield\", or character highlights like \"KWON with AWP\".\n   - Craft a concise yet intriguing title that captures the essence of the video content and piques interest. Keep in mind that all of the content generated is from the Discord server \"Blink\" so something along the lines of \"Tales from Blink: [Character] is on a roll\" is very funny\n\n2. **Description Creation**:\n   - Summarize the main narrative or themes conveyed through the lyrics and scene description.\n   - Include elements such as character highlights, digital landscapes, and key action points from the lyrics.\n   - Engage the audience by hinting at the excitement and visual spectacle of the video.\n\n3. **Tag Selection**:\n   - Extract relevant tags from prominent elements like character names, technology or gaming terms, and thematic concepts.\n   - Use a mix of broader terms and video-specific keywords to enhance discoverability.\n\n# Output Format\n\n- **Title**: A single line, ideally 50-60 characters.\n- **Description**: A short paragraph, approximately 100-200 words.\n- **Tags**: A list of 5-10 relevant keywords/phrases, separated by commas.\n\n# Example\n\n**Title**: \"Neon Nights in Digital Battlefield - KWON's Tale\"\n\n**Description**: \nJoin KWON in a thrilling adventure across a luminous digital battlefield where technology and nature collide. Experience the synergy of city lights and tactical prowess, as KWON wields the brilliant green energy of his futuristic AWP sniper rifle. With allies like Reyna and Phoenix at his side, and battles set against the backdrop of vibrant landscapes, this video captures the thrill of gaming skill and team coordination in stunning visual style. Don't miss the dynamic action and strategic gameplay as KWON and his friends take on new challenges and shine against all odds.\n\n**Tags**: KWON, AWP sniper, digital battlefield, gaming skill, teamwork, cyber landscape, Pramit, synergy, city lights, tactical gameplay\n\n# Notes\n\n- Ensure the title is SEO-friendly and likely to attract viewers.\n- The description should seamlessly integrate with YouTube's formatting constraints and viewer expectations.\n- Consider the balance between specificity and broad appeal when selecting tags."
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "(Verse 1) Pramit said, \"Where do you work? Oh, Wall Street Company!\" Everyone's talking, so much synergy Neon lights and city nights Deadlock season's the latest hype (Chorus) Let's go shower, flash and tower 30 minutes to power, this is our hour Grab the orb, shower scene Cypher’s here, oh how we've been (Verse 2) Sova, Reyna, Phoenix too Headphones on backwards, what a view Jon says, \"Try not to die\" Lucy dings Phoenix, Naiyo in the sky (Chorus) Let's go shower, flash and tower 30 minutes to power, this is our hour Pramit with the AWP, nothing sour Holding angles, gaming flower (Bridge) What’s that dog doing? Jimmy wants to know Reyna’s lurking, sneaky like a Poe Get in position, all set, let's roll Round and round, capture the soul (Outro) KAYPramitO, take us out In this game, we’re never in doubt Shower's clear, shine bright, no fear We play for fun, year after year\n\n[Setting: Inside a dimly lit gaming arena, the atmosphere is tense and focused with the sound of muted keyboard clacks echoing throughout. The room is filled with large screens projecting real-time game statistics.] - *Character Focus: JIMMY* crouches behind his digital console, whispering strategies into his microphone, wearing a tactical helmet with digital camouflage patterns. His eyes are locked on the screens, illustrating dedication and concentration, reflecting “Jon says, 'Try not to die.” A motion-capture display of Phoenix looms over him on a screen, highlighting his cautionary cues. - The scene aligns with the urgency captured in the verse, pulling viewers into the gaming action.\n\n[Setting: An abstract rendition of a shower room, swirling with mist and brilliant blue light that shifts like water. Suspenseful music heightens a sense of impending action.] - *Character Focus: NAIYO* stands in the center, smirking and twirling a glowing blue orb that emits tiny lightning bolts. His outfit appears like liquid metal, shifting colors as he moves—giving an ethereal feel tied to the \"flash and tower\" section of the chorus. - This scene brings the chorus into focus, as the background oscillates between showers and storms, emphasizing strategy and excitement.\n\n[Setting: A digital battlefield, representing a blend of a video game and a vibrant outdoor landscape. The ground sparkles as if lined with luminous circuits.] - *Character Focus: JUJU* holds a futuristic AWP sniper rifle, its energy glowing a brilliant green. He moves with cat-like agility, highlighted by his streamlined armor, merging cyber and tactical styles. His name appears on-screen next to his avatar in a heads-up display detailing his moves. - The imagery of \"Pramit with the AWP...gaming flower\" resonates visually as nature and technology blend, driving forward the themes of conflict and skill."
        }
      ]
    },
    {
      "role": "assistant",
      "content": [
        {
          "type": "text",
          "text": "{\"title\":\"Blink Slander: Gaming Shenanigans with JUJU & Friends\",\"description\":\"Dive into the electrifying world of gaming with 'Blink Slander' as we follow JUJU, NAIYO, and the rest of the squad in their latest high-stakes challenge! Inside a dimly lit arena, the tension is palpable as strategies unfold, whispers echo, and digital magic comes alive. With neon lights illuminating the battlefield and a swirling shower of energy, watch JUJU dominate the scene with his AWP sniper and NAIYO channel his inner power. Each moment is packed with excitement, humor, and unexpected twists—perfect for adrenaline junkies and gaming lovers alike. Join us for laughter, teamwork, and non-stop action as we embark on a wild ride through the vibrant world of gaming!\",\"tags\":[\"gaming\",\"Blink Slander\",\"JUJU\",\"AWP sniper\",\"NAIYO\",\"Cypher\",\"Sova\",\"Phoenix\",\"teamwork\",\"digital battlefield\"],\"categoryId\":\"20\"}"
        }
      ]
    }
  ],
  temperature=1,
  max_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  response_format={
    "type": "json_schema",
    "json_schema": {
      "name": "youtube_video_metadata",
      "strict": True,
      "schema": {
        "type": "object",
        "properties": {
          "title": {
            "type": "string",
            "description": "The title of the YouTube video."
          },
          "description": {
            "type": "string",
            "description": "A detailed description of the YouTube video."
          },
          "tags": {
            "type": "array",
            "description": "A list of tags relevant to the video for better discoverability.",
            "items": {
              "type": "string"
            }
          },
          "categoryId": {
            "type": "string",
            "description": "The category ID of the video, e.g., '22' for 'People & Blogs'."
          }
        },
        "required": [
          "title",
          "description",
          "tags",
          "categoryId"
        ],
        "additionalProperties": False
      }
    }
  }
)

In [14]:
raw_json = response.choices[0].message.content

In [15]:
import json
json.loads(raw_json)


{'title': "Blink Slander: JUJU's AWP Domination & Shenanigans!",
 'description': "Join us in the world of 'Blink Slander' where excitement and action collide! In this thrilling gaming adventure, watch JUJU as he showcases his epic skills with a glowing AWP sniper rifle, revealing stunning shots amidst a digital battlefield filled with vibrant landscapes. The crew—including NAIYO with his enchanting orb, and a surprise cameo from the stealthy Reyna—embraces the hype in a neon-lit arena, strategizing their way to victory. With catchy beats, hilarious moments, and intense gameplay, this video captures the essence of teamwork and skill in the ultimate gaming showdown. Don’t miss out on the laughter, intensity, and mesmerizing visuals as we strive to capture the soul of gaming!",
 'tags': ['JUJU',
  'AWP sniper',
  'gaming journey',
  'Blink Slander',
  'NAIYO',
  'cyber landscape',
  'Reyna',
  'Sova',
  'Phoenix',
  'team coordination'],
 'categoryId': '20'}